In [1]:
print("Hello World!")

Hello World!


In [9]:
import snowflake.connector
import os
import pandas as pd
import dotenv as dot
# from dotenv import load_dotenv

DATABASE_SCHEMA = "PLAYGROUND.DATATHON"

def get_snowflake_connection():
    """
    Create a connection to Snowflake using credentials from .env file
    """
    # Load environment variables
    dot.load_dotenv()
    
    # Get connection parameters from environment variables
    conn = snowflake.connector.connect(
        account=os.getenv("SNOWFLAKE_ACCOUNT"),
        user=os.getenv("SNOWFLAKE_USER"),
        password=os.getenv("SNOWFLAKE_PASSWORD"),
        role=os.getenv("SNOWFLAKE_ROLE"),
        warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
        database=os.getenv("SNOWFLAKE_DATABASE"),
        schema=os.getenv("SNOWFLAKE_SCHEMA")
    )
    
    return conn

def query_to_df(query):
    """
    Execute a query and return the results as a pandas DataFrame
    """
    conn = get_snowflake_connection()
    cursor = conn.cursor()
    cursor.execute(query)

    # Get column names
    columns = [col[0] for col in cursor.description]

    # Fetch all rows and convert to list of dictionaries
    results = [dict(zip(columns, row)) for row in cursor.fetchall()]

    cursor.close()
    conn.close()

    return pd.DataFrame(results)


# Example status_df 
status = query_to_df(f"SELECT * FROM {DATABASE_SCHEMA}.status")
print(status.head())

   STATUSID        STATUS
0         1      Finished
1         2  Disqualified
2         3      Accident
3         4     Collision
4         5        Engine
